<a href="https://colab.research.google.com/github/fortytwo09/webrtc-lab/blob/master/sentiqTweepyTestHome.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install ast
!pip install tweepy==3.10.0
!pip install gspread
!pip install schedule

In [325]:
from ast import keyword
import tweepy
import time
import random
import re
import gspread
import pprint
import schedule
import time
import math
import json
import pandas as pd
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [339]:
CONSUMER_KEY = "2sHvmDN9a44FxhBS1mgXu2OsL"
CONSUMER_SECRET = "os4HaZLO5GgCGTPMrJpjVugf4UuTwkPqyudhqBBQ3cukVBPuqc"
ACCESS_KEY = "1504525510445252608-srWdbA57nL8FBc7FjQKX4CGfp3VpT3"
ACCESS_SECRET = "7LTQ8QOBSD6cqcYf29ksK4Aisp12aQ887uzbF258vXHMr"

In [338]:
def refresh_auth():
    auth  = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
    auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)
    api = tweepy.API(auth, wait_on_rate_limit=True)
    return api
  
def roll_dice(mention_keyword):
    dice_info = []
    dice_result_list = []

    if len(mention_keyword) > 1: # 넘겨받은 매개변수의 길이가 1 초과라면 (예시 : ['다이스', '1d2'] 이므로 길이는 2다)
        second_keyword = mention_keyword[1].strip() # 두번째 키워드를 보정한다. strip()은 앞뒤 공백을 지우는 함수다. (ex: ' 1d2 ' 였다면 '1d2'로 만든다)
        
        if 'd' in second_keyword: # 두번째 키워드 중에 d가 있다면
            dice_info = second_keyword.split('d') # d를 기준으로 나누고 dice_info에 저장 (ex: '1d2' -> ['1', 'd', '2'])
        elif 'D' in second_keyword: # 두번째 키워드 중에 D가 있다면
            dice_info = second_keyword.split('D') # D를 기준으로 나누고 dice_info에 저장 (ex: '1D2' -> ['1', 'D', '2'])
        
        if len(dice_info) == 2: # D 앞뒤로 숫자 있나확인
            try:
                dice_f_num = int(dice_info[0]) # 첫번째 값은 '몇 번 굴릴것'인가 
                dice_s_num = int(dice_info[1]) # 두번째 값은 '몇 면체'인가
                for _ in range(dice_f_num): # 첫번째 값만큼 반복
                    dice = random.randrange(1, dice_s_num+1) # 1 ~ 몇면체 사이의 값을 랜덤하게 뽑아서 dice에 저장
                    dice_result_list.append(dice) # tmp_list에 굴린 다이스를 순서대로 추가 
                return (dice_f_num, dice_s_num, dice_result_list) # 몇 번, 몇 면체, 총합, 다이스 숫자를 담아서 리턴
                # return dice_result_list
            except:
                print("에러 발생")
                pass

    print("! 다이스 키워드가 잘못됐습니다")    
    return(-1, -1, -1)

In [ ]:
def sheetInsert(targetsheet, testval):
    testval.append( str(time.strftime("%Y-%m-%d %H:%M:%S")) ) #updat_date
    testval.append( str(time.strftime("%Y-%m-%d %H:%M:%S")) ) #reg_date
    testval.insert( 0, "" ) # blank
    print(type(testval))
    targetsheet.append_row(testval)

In [340]:
api = refresh_auth()
print("api Refresh_auth SUCCESS")

api Refresh_auth SUCCESS


In [269]:
def val_for_diceRoll(StrVal):
  totalVal = 0
  rollCount = 1
  if 'd' in StrVal:
    rollCount = int(StrVal[:StrVal.find('d')])
    StrVal = StrVal[int(StrVal.find('d'))+1:]

  diceValList = StrVal.split(sep='-')
  for j in range(rollCount):
    totalVal += random.randrange(int(diceValList[0]), int(diceValList[1])+1)
  return totalVal

def val_for_randomChoice(randomStrVal):
  TestValVals = randomStrVal[re.search(r"[:]",randomStrVal).start()+1 : ].split(sep='/')
  randomVal = random.choice(TestValVals)
  return randomVal

def val_for_percentageChoice(randomStrVal):
  TestValVals = randomStrVal[re.search(r"[:]",randomStrVal).start()+1 : ].split(sep='/')
  percentageList = eval(TestValVals[0])
  percentageStrList = eval(TestValVals[1])
  randomVal = random.choices(population=percentageStrList, weights=percentageList)
  return str(randomVal).lstrip("['").rstrip("']")+" "

def checkRollExists(inputTxt):
  rollVal = re.search(r"[\{]+[^\{|^\}]+[\}]{1}", inputTxt)

  if rollVal is not None :
    rollData = inputTxt[rollVal.start():rollVal.end()-1]
    TestValFunc = rollData[: re.search(r"[:]",rollData).start()]
    TestValResult = ""

    if 'random' in TestValFunc: #현재 random밖에 없음
      choiceVal = val_for_randomChoice(rollData)
      if '-' in choiceVal : #n-m의 다이스일 경우
        rollData = val_for_diceRoll(choiceVal)
      else : #문자열 값
        rollData = choiceVal
    
    if 'dice' in TestValFunc:
      choiceVal = val_for_randomChoice(rollData)
      rollData = val_for_diceRoll(choiceVal)

    if 'percentage' in TestValFunc:
      rollData = val_for_percentageChoice(rollData)

    TestValResult = inputTxt[:rollVal.start()-1]+" "+str(rollData)+inputTxt[rollVal.end():]
    return checkRollExists(TestValResult) #재귀
  else :
    return inputTxt

In [270]:
textTxt = "누군가의 인영을 [발견]하기 쉬운 것 같다. {percentage: [0.2, 0.4, 0.8]/['2할', '4할', '8할']}확률. {dice:10-15}개의 바람개비가 놓여있다. {random:바람이 시원하다. 'Q코인 {random:8-12/1-3}개' 획득./철조망 너머의 광경이 보인다.} {random:[자물쇠 달린 철조망]/[하트 조각상]}을 볼까? {random:[스카이 데크]/[타워 1층]}에 가도 좋고."
# ndx-y >> n번 x부터 y값으로
# {percentage: [0.2, 0.4, 0.8]/['2할', '4할', '8할']} >> 꼭 순서 맞춰주고 문자가 들어가는 경우 ' ' 내에 넣기
print(checkRollExists(textTxt))

누군가의 인영을 [발견]하기 쉬운 것 같다. 2할 확률. 13개의 바람개비가 놓여있다. 철조망 너머의 광경이 보인다. [자물쇠 달린 철조망]을 볼까? [타워 1층]에 가도 좋고.


In [343]:
auth.authenticate_user()
gc = gspread.authorize(GoogleCredentials.get_application_default())
sh = gc.open('sentiq')

In [345]:
print('트윗 확인 중...', flush=True)
api = refresh_auth()
last_seen_id = sh.get_worksheet(1).acell('A1').value
mentions = api.mentions_timeline(last_seen_id,tweet_mode='extended')
print()

for mention in reversed(mentions):
  entities = mention.entities['user_mentions'] # 멘션 내 언급된 대상자들
  print(str(mention.id) + ' - ' + mention.full_text, flush=True)
  last_seen_id = mention.id

user = api.get_user(mention.user.screen_name)
name = user.name

######### sheet 선언
memberSheet = sh.sheet1
memberCell = memberSheet.find(mention.user.screen_name)
ExpSheet = sh.worksheet("exp")
logSheet = sh.worksheet("log")
ExaminLv1Sheet = sh.worksheet("examine_lv1")

if '[조사]' in mention.full_text.lower():
  examArr = ExaminLv1Sheet.col_values(2)
  for lv1 in examArr:
    if lv1 in mention.full_text.lower():
      ExamLv1Cell = ExaminLv1Sheet.find(lv1)
      print("조사 IN")

트윗 확인 중...

1504883252833652739 - @STQ_diceB [나고야테스트]


In [ ]:
def reply_to_tweets():
  print('트윗 확인 중...', flush=True)
  api = refresh_auth()
  sh.get_worksheet(1)
  last_seen_id = (sh.get_worksheet(1)).cell(sh.get_worksheet(1))
  mentions = api.mentions_timeline(last_seen_id,tweet_mode='extended')
  print()

  for mention in reversed(mentions):
    entities = mention.entities['user_mentions'] # 멘션 내 언급된 대상자들
    print(str(mention.id) + ' - ' + mention.full_text, flush=True)
    last_seen_id = mention.id

  user = api.get_user(mention.user.screen_name)
  name = user.name

  ######### sheet 선언
  memberSheet = sh.sheet1
  memberCell = memberSheet.find(mention.user.screen_name)
  ExpSheet = sh.get_worksheet(1)
  logSheet = sh.get_worksheet(3)
  delta = 1 #증가시킬 숫자

  ########## 유저 정보 선언
  examineTimes = int(memberSheet.cell(memberCell.row, 4).value) #횟수 감산 값
  # qChipNum = int(memberSheet.cell(memberCell.row, 5).value) # Q칩
  attackDice = (memberSheet.cell(memberCell.row, 6).value) # 공격다이스
  attackStat = int(memberSheet.cell(memberCell.row, 7).value) # 공격스탯
  depenceDice = (memberSheet.cell(memberCell.row, 8).value) # 방어다이스
  depenceStat = int(memberSheet.cell(memberCell.row, 9).value) # 방어스탯
  hpStat = int(memberSheet.cell(memberCell.row, 10).value) # 체력
  sanStat = int(memberSheet.cell(memberCell.row, 11).value) # 정신지수

  ## TO DO :: 개편
  # ExpSheet에서 키워드 라인 값 배열로 가져오기
  expArr = ExpSheet.col_values(2) #['', '전체 키워드', '[나고야조사]', '[조사중지]', '[공격]', '[방어]', '[Hp정산]']
  # switch로 keyword매칭
  ## TO DO :: END

  if '[조사]' in mention.full_text.lower():
   print("조사 IN")
  else:
    print("NOT FIND COMMAND")

  # schedule.every().days.at("07:00").do(printtime1(mention)
  store_last_seen_id(last_seen_id, FILE_NAME) # 완료될 경우(오류가 안 나고 제대로 된 실행=돈 들어올 때) last_id로 업데이트

reply_to_tweets()
print("__END__")